In [1]:
import pandas as pd

In [2]:
IN_DATA_PATH = r'C:\Data Sciences\Data\inputs'
OUT_DATA_PATH = r'C:\Data Sciences\Data\outputs'

In [10]:
pub_health_train = pd.read_csv(IN_DATA_PATH + r"\train.tsv", sep='\t')
pub_health_test = pd.read_csv(IN_DATA_PATH + r"\test.tsv", sep='\t')

In [12]:
pub_health_train = pub_health_train[pub_health_train['label'] != 'snopes']
pub_health_train = pub_health_train[['main_text','label']]
pub_health_train = pub_health_train.dropna(subset=['main_text', 'label'])

pub_health_train.head()

,main_text,label
0,"""Hillary Clinton is in the political crosshair...",false
1,While the financial costs of screening mammogr...,mixture
2,The news release quotes lead researcher Robert...,mixture
3,"The story does discuss costs, but the framing ...",true
4,"""Although the story didn’t cite the cost of ap...",true


In [13]:
pub_health_test = pub_health_test[['main_text','label']]
pub_health_test = pub_health_test.dropna(subset=['main_text', 'label'])